In [ ]:
import sys
from pathlib import Path
srcdir = Path("../..").resolve()
print(f"Adding {srcdir} to sys.path, this is necessary to import from src")
sys.path.insert(0, str(srcdir))
print(sys.path)

from src.settings import SearchSpace


First:
- go to the terminal
- go to the ML22 folder, use `cd ~/ML22` or `z ML` if necessary
- if you are planning on closing the laptop to let this run by itself, start a tmux session by giving the command `tmux`. You can name the session with `ctrl+b $`, leave the tmux with `ctrl+b` d, access it again with `tmux a -t 0` or replace the 0 with the name you gave it. See [tmux](https://github.com/tmux/tmux/wiki/Getting-Started) for more info.
- in that folder is the file `hypertune.py`. Run it with `poetry run python hypertune.py`
- In the hypertune.py file, a tune_dir is specified: `models/ray`. We will check the contents of that folder after the hypertune finished. You can also use tensorboard to check the results. 
- this will take some time. I clocked 1901 seconds (30 minutes) on the VM

 0.0992908 |           119 |            3 |     26 |         576.744  |    0 |   0.942187

In [ ]:
tune_dir = Path("../../models/ray").resolve()
tune_dir.exists()


In [ ]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)

analysis = ExperimentAnalysis(tune_dir)


So, we find some info:

In [ ]:
analysis.results_df.columns


Let's focus on the parameters we wanted to tune.

In [ ]:
import plotly.express as px

plot = analysis.results_df
select = ["Accuracy", "config/hidden_size", "config/dropout", "config/num_layers"]
p = plot[select].reset_index().dropna()


Let's sort by accuracy

In [ ]:
p.sort_values("Accuracy", inplace=True)

Make a parallel plot

In [ ]:
px.parallel_coordinates(p, color="Accuracy")


Get the best trial

In [ ]:
analysis.get_best_trial(metric="test_loss", mode="min")


The top ten

In [ ]:
p[-10:]


Or the best config

In [ ]:
analysis.get_best_config(metric="Accuracy", mode="max")
